# Nombre: Marco Sebastian Ocampo
# Laboratorio 3
## Curso: Data Wrangling
## Fecha: 21 de agosto del 2023

### PARTE 1 (15 pts): Utilizando la base de datos vista en clase, conteste las siguientes preguntas.

In [2]:
### Librerias a usar

import pandas as pd
import mysql.connector

In [3]:
### Conexión a la base de datos de empleados

from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

host = "relational.fit.cvut.cz"
user = "guest"
pwd = "relational"
connection = create_connection(host_name = host, user_name =user ,user_password = pwd)

Connection to MySQL DB successful


1.	Obtener el nombre, apellido, salario, título y vigencia del mismo para el empleado 12557. Su resultado se debe de ver de la siguiente manera:

In [6]:
PREGUNTA1 = """
SELECT *
FROM(
    SELECT
        A.emp_no,
        A.first_name,
        A.last_name,
        C.title,
        B.salary,
        B.from_date,
        B.to_date
    FROM employee.employees A
    LEFT JOIN employee.salaries B ON B.emp_no = A.emp_no
    LEFT JOIN employee.titles C ON C.emp_no = A.emp_no
    GROUP BY 1,2,3,4,5,6,7
    ) A
WHERE emp_no = 12557
"""

pd.read_sql(PREGUNTA1,connection)

,emp_no,first_name,last_name,title,salary,from_date,to_date
0,12557,Stafford,Kopetz,Senior Engineer,41840,1988-03-20,1989-03-20
1,12557,Stafford,Kopetz,Senior Engineer,46021,1989-03-20,1990-03-20
2,12557,Stafford,Kopetz,Senior Engineer,48699,1990-03-20,1991-03-20
3,12557,Stafford,Kopetz,Senior Engineer,51351,1992-03-19,1993-03-19
4,12557,Stafford,Kopetz,Senior Engineer,51610,1991-03-20,1992-03-19
5,12557,Stafford,Kopetz,Senior Engineer,51660,1993-03-19,1994-03-19
6,12557,Stafford,Kopetz,Senior Engineer,52878,1994-03-19,1995-03-19
7,12557,Stafford,Kopetz,Senior Engineer,54053,1995-03-19,1996-03-18
8,12557,Stafford,Kopetz,Senior Engineer,57950,1996-03-18,1997-03-18
9,12557,Stafford,Kopetz,Senior Engineer,58055,1997-03-18,1998-03-18


2.	Encontrar el último departamento en el que trabajó el empleado con la siguiente información del mismo: nombre, apellido, número de empleado, fecha de nacimiento, código de departamento, nombre del departamento, fecha cuando inició su trabajo en ese departamento y la fecha cuando concluyó su trabajo

In [8]:
PREGUNTA2 = """
SELECT * 
FROM
    (SELECT
        A.emp_no,
        A.first_name,
        A.last_name,
        A.birth_date,
        B.dept_no,
        C.dept_name,
        B.from_date,
        B.to_date,
        ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY B.from_date DESC) AS sec
    FROM employee.employees A
    LEFT JOIN employee.dept_emp B ON B.emp_no = A.emp_no
    LEFT JOIN employee.departments C ON C.dept_no = B.dept_no
    GROUP BY 1,2,3,4,5,6,7,8)A
WHERE emp_no = 12557
"""
pd.read_sql(PREGUNTA2,connection)

### El último departamento en el que estuvo este empleado es el departamento con el código d004

,emp_no,first_name,last_name,birth_date,dept_no,dept_name,from_date,to_date,sec
0,12557,Stafford,Kopetz,1954-01-01,d004,Production,1994-09-23,9999-01-01,1
1,12557,Stafford,Kopetz,1954-01-01,d006,Quality Management,1988-03-20,1994-09-23,2


3.	Encontrar el salario máximo de cada empleado utilizando la función Partition By con toda la información del empleado y de su salario

In [6]:
PREGUNTA3 = """
SELECT * FROM
(SELECT
        A.emp_no,
        A.first_name,
        A.last_name,
        A.birth_date,
        B.dept_no,
        C.dept_name,
        B.from_date as date01,
        B.to_date as date02,
        D.from_date,
        D.to_date,
        D.salary,
        ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY D.salary DESC) AS sec
FROM employee.employees A
LEFT JOIN employee.dept_emp B ON B.emp_no = A.emp_no
LEFT JOIN employee.departments C ON C.dept_no = B.dept_no
LEFT JOIN employee.salaries D ON D.emp_no = A.emp_no AND(D.to_date <= B.to_date AND D.from_date >= B.from_date)
GROUP BY 1,2,3,4,5,6,7,8,9,10,11) A
WHERE sec = 1
LIMIT 5
"""

pd.read_sql(PREGUNTA3,connection)

,emp_no,first_name,last_name,birth_date,dept_no,dept_name,date01,date02,from_date,to_date,salary,sec
0,10001,Georgi,Facello,1953-09-02,d005,Development,1986-06-26,9999-01-01,2002-06-22,9999-01-01,88958,1
1,10002,Bezalel,Simmel,1964-06-02,d007,Sales,1996-08-03,9999-01-01,2001-08-02,9999-01-01,72527,1
2,10003,Parto,Bamford,1959-12-03,d004,Production,1995-12-03,9999-01-01,2000-12-01,2001-12-01,43699,1
3,10004,Chirstian,Koblick,1954-05-01,d004,Production,1986-12-01,9999-01-01,2001-11-27,9999-01-01,74057,1
4,10005,Kyoichi,Maliniak,1955-01-21,d003,Human Resources,1989-09-12,9999-01-01,2001-09-09,9999-01-01,94692,1
5,10006,Anneke,Preusig,1953-04-20,d005,Development,1990-08-05,9999-01-01,2000-08-02,2001-08-02,60098,1
6,10007,Tzvetan,Zielinski,1957-05-23,d008,Research,1989-02-10,9999-01-01,2002-02-07,9999-01-01,88070,1
7,10008,Saniya,Kalloufi,1958-02-19,d005,Development,1998-03-11,2000-07-31,2000-03-10,2000-07-31,52668,1
8,10009,Sumant,Peac,1952-04-19,d006,Quality Management,1985-02-18,9999-01-01,2001-02-14,2002-02-14,94443,1
9,10010,Duangkaew,Piveteau,1963-06-01,d006,Quality Management,2000-06-26,9999-01-01,2001-11-23,9999-01-01,80324,1


### Misterio en SQL City


In [20]:
import pandas as pd
import sqlite3 as sq
db = sq.connect('sql-murder-mystery.db')

In [28]:
STR = db.execute("SELECT name, sql FROM sqlite_master WHERE type = 'table';")
for i in STR:
    print('Tabla:',i[0],'\nStr:',i[1])

Tabla: crime_scene_report 
Str: CREATE TABLE crime_scene_report (
        date integer,
        type text,
        description text,
        city text
    )
Tabla: drivers_license 
Str: CREATE TABLE drivers_license (
        id integer PRIMARY KEY,
        age integer,
        height integer,
        eye_color text,
        hair_color text,
        gender text,
        plate_number text,
        car_make text,
        car_model text
    )
Tabla: person 
Str: CREATE TABLE person (
        id integer PRIMARY KEY,
        name text,
        license_id integer,
        address_number integer,
        address_street_name text,
        ssn integer,
        FOREIGN KEY (license_id) REFERENCES drivers_license(id)
    )
Tabla: facebook_event_checkin 
Str: CREATE TABLE facebook_event_checkin (
        person_id integer,
        event_id integer,
        event_name text,
        date integer,
        FOREIGN KEY (person_id) REFERENCES person(id)
    )
Tabla: interview 
Str: CREATE TABLE interview

### PISTA: Recuerdo que el asesinate fue hecho el 15 de enero del 2018 en SQL City 

In [41]:
PISTA1 ="""SELECT * 
FROM crime_scene_report
WHERE type = 'murder'
AND city = 'SQL City'
AND date = '20180115'
"""
pd.read_sql(PISTA1, db)

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [44]:
PISTA2 ="""SELECT * 
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER By address_number DESC
LIMIT 10
"""
pd.read_sql(PISTA2, db)

### El primer testigo se llama Morty Shapiro

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
2,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
3,73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
4,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
5,19420,Cody Schiel,890431,3524,Northwestern Dr,947110049
6,93509,Emmitt Aceuedo,916706,3491,Northwestern Dr,979073160
7,87456,Leonora Wolfsberger,215868,3483,Northwestern Dr,565203106
8,36378,Freddie Ellzey,267882,3449,Northwestern Dr,474117596
9,53076,Boris Bijou,664914,3327,Northwestern Dr,401191868


In [56]:
PISTA2 ="""SELECT * 
FROM person
WHERE address_street_name = 'Franklin Ave'
AND name LIKE '%annabel%'
"""
pd.read_sql(PISTA2, db)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [59]:
PISTA3 ="""SELECT * 
FROM person
WHERE id IN (14887, 16371)
"""
pd.read_sql(PISTA3, db)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [64]:
PISTA4 ="""SELECT * 
FROM interview
WHERE person_id IN (14887, 16371)
"""
pd.read_sql(PISTA4, db)


,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


In [84]:
Testigo1 ="""SELECT * 
FROM interview
WHERE person_id IN (14887)
"""
pd.set_option('display.max_colwidth', None)
pd.read_sql(Testigo1, db)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [87]:
Testigo2 ="""SELECT * 
FROM interview
WHERE person_id IN (16371)
"""
pd.set_option('display.max_colwidth', None)
pd.read_sql(Testigo2, db)

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [93]:
PISTA5 ="""SELECT * 
FROM drivers_license as dl
INNER JOIN person as p on dl.id = p.license_id
WHERE plate_number LIKE '%H42%'
"""
pd.read_sql(PISTA5, db)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,643735,67,62,blue,grey,female,52H42S,Chevrolet,Tahoe,50583,Senaida Gianelli,643735,1109,Holloway St,147534844
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima,51739,Tushar Chandra,664760,312,Phi St,137882671
2,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
3,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius,78193,Maxine Whitely,183779,110,Fisk Rd,137882671
4,817202,42,62,blue,red,female,5H427P,Pontiac,Trans Sport,90979,Rosalba Devery,817202,3032,Spoon Hill Dr,188925373


In [96]:
PISTA6 ="""SELECT p.* 
FROM drivers_license as dl
INNER JOIN person as p on dl.id = p.license_id
INNER JOIN get_fit_now_member as getfit on p.id = getfit.person_id
WHERE plate_number LIKE '%H42%'
"""
pd.read_sql(PISTA6, db)

,id,name,license_id,address_number,address_street_name,ssn
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [101]:
cursor = db.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
db.commit()
var = pd.read_sql("SELECT value FROM solution", db)
var.iloc[0,0]


"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [103]:
Testigo3 ="""SELECT * 
FROM interview
WHERE person_id IN (67318)
"""
pd.read_sql(Testigo3, db)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [109]:
Testigo3 ="""SELECT * 
FROM drivers_license
WHERE hair_color = 'red'
AND height BETWEEN 65 AND 67
AND car_model = "Model S"
AND gender = 'female'
"""
pd.read_sql(Testigo3, db)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [113]:
Testigo4 ="""SELECT * 
FROM drivers_license as dl
INNER JOIN person on dl.id
INNER JOIN facebook_event_checkin as fb on fb.person_id = person.id
WHERE hair_color = 'red'
AND height BETWEEN 65 AND 67
AND car_model = "Model S"
AND gender = 'female'
"""
pd.read_sql(Testigo4, db)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn,person_id,event_id,event_name,date
0,918773,48,65,black,red,female,917UU3,Tesla,Model S,28508,Raylene Goldsby,531984,1696,S Holland St,817820356,28508,5880,Nudists are people who wear one-button suits.\n,20170913
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S,28508,Raylene Goldsby,531984,1696,S Holland St,817820356,28508,5880,Nudists are people who wear one-button suits.\n,20170913
2,202298,68,66,green,red,female,500123,Tesla,Model S,28508,Raylene Goldsby,531984,1696,S Holland St,817820356,28508,5880,Nudists are people who wear one-button suits.\n,20170913
3,918773,48,65,black,red,female,917UU3,Tesla,Model S,63713,Josh Cunnane,990581,3445,Evers Way,334859137,63713,3865,but that's because it's the best book on anything for the layman.\n,20171009
4,291182,65,66,blue,red,female,08CM64,Tesla,Model S,63713,Josh Cunnane,990581,3445,Evers Way,334859137,63713,3865,but that's because it's the best book on anything for the layman.\n,20171009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60028,291182,65,66,blue,red,female,08CM64,Tesla,Model S,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,67318,4719,The Funky Grooves Tour,20180115
60029,202298,68,66,green,red,female,500123,Tesla,Model S,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,67318,4719,The Funky Grooves Tour,20180115
60030,918773,48,65,black,red,female,917UU3,Tesla,Model S,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,67318,1143,SQL Symphony Concert,20171206
60031,291182,65,66,blue,red,female,08CM64,Tesla,Model S,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,67318,1143,SQL Symphony Concert,20171206


In [ ]:
cursor = db.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
db.commit()
var = pd.read_sql("SELECT value FROM solution", db)
var.iloc[0,0]